# prossesing the data

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


def load_all_txt(input_dir):
    all_text = ""

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                all_text += f.read() + "\n"

    return all_text

In [2]:
input_directory = r"/kaggle/input/chess-analyse-books"
full_text = load_all_txt(input_directory)
print(len(full_text))

5556076


# load and test model

In [3]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found e

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch

model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float32,
    device_map="auto"
)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [5]:
class StopAtSentence(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Check last token
        last_token = input_ids[0][-1].item()
        # Stop if EOS or common sentence-ending token (".", "?", "!")
        if last_token == tokenizer.eos_token_id:
            return True
        decoded = tokenizer.decode([last_token], skip_special_tokens=True)
        if decoded.strip() in {".", "?", "!"}:
            return True
        return False

def chat_qwen3_direct(prompt: str, max_new_tokens: int = 256) -> str:
    messages = [{"role": "user", "content": prompt}]
    
    # Apply chat template WITHOUT thinking
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # 🔥 Disable thinking for factual answers
    )
    
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,      # Lower = more focused
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # stopping_criteria=StoppingCriteriaList([StopAtSentence()])  # Optional
        )
    
    # Extract only the new text
    input_len = inputs.input_ids.shape[1]
    response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
    
    # Clean: remove any trailing incomplete sentence
    for end in [".", "?", "!"]:
        if end in response:
            response = response[:response.rfind(end)+1]
            break
    else:
        # No sentence end? Return first line or truncate
        response = response.split("\n")[0].strip()
    
    return response.strip()

In [6]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt, max_new_tokens=32768)
print("💬 Answer:", answer)

💬 Answer: Controlling the center is crucial in chess because it provides a strategic advantage by allowing players to control key pieces and move their pieces more effectively. This control also helps in creating a flexible and dynamic game, as the center is a common target for both attackers and defenders.


# fine tune the model

In [7]:
def chunk_text(text, tokenizer, max_length):
    # Use batched tokenization to avoid huge tensors
    tokens = tokenizer(
        text,
        return_tensors="pt",
        add_special_tokens=False,  # Avoid extra BOS/EOS in middle of book
        truncation=False,
        verbose=False
    )["input_ids"][0]

    chunks = []
    # Use full chunks only (avoid tiny trailing fragments)
    for i in range(0, len(tokens) - max_length + 1, max_length):
        chunk = tokens[i:i + max_length].tolist()
        chunks.append({"input_ids": chunk})
    
    # Optional: add last partial chunk if long enough
    remainder = len(tokens) % max_length
    if remainder > 64:  # only if >64 tokens
        chunks.append({"input_ids": tokens[-remainder:].tolist()})
    
    return chunks

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

max_length = 256

data = chunk_text(full_text, tokenizer, max_length)
dataset = Dataset.from_list(data)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,                              
    pad_to_multiple_of=8                    
)

In [9]:
training_args = TrainingArguments(
    output_dir="./chess-qwen",
    run_name="chess-qwen-run-1",

    
    per_device_train_batch_size=2,    
    gradient_accumulation_steps=16,   

   
    num_train_epochs=3,
    learning_rate=2e-4,                
    fp16=True,                      
    bf16=False,                        

    
    save_steps=2000,                   
    logging_steps=200,                  
    save_total_limit=1,                 

    # 🚀 Other speedups
    optim="adamw_torch",                
    report_to="none",                  
    dataloader_num_workers=4,           
    remove_unused_columns=True,
)


# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

In [10]:
trainer.train()


model.save_pretrained("./chess-qwen-adapted")
tokenizer.save_pretrained("./chess-qwen-adapted")

Step,Training Loss
200,2.783500
400,2.091400
600,1.545300


('./chess-qwen-adapted/tokenizer_config.json',
 './chess-qwen-adapted/special_tokens_map.json',
 './chess-qwen-adapted/chat_template.jinja',
 './chess-qwen-adapted/vocab.json',
 './chess-qwen-adapted/merges.txt',
 './chess-qwen-adapted/added_tokens.json',
 './chess-qwen-adapted/tokenizer.json')

In [11]:
del trainer
del model
del data_collator
del dataset
del tokenizer

torch.cuda.empty_cache()

# test the adapted model

In [12]:
model_name = "/kaggle/working/chess-qwen-adapted"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [13]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt, max_new_tokens=256)
print("💬 Answer:", answer)

💬 Answer: legates: The control of the center is a major factor in the success of a player’s opening strategy and the maintenance or even the growing of a winning position. It is a major factor in the development of the pieces and the coordination of the forces, and it is a major part of the strategic concept in the opening.


# Quantizing the model

In [14]:
!pip install sentencepiece
!pip install "llama-cpp-python>=0.2.0"
!git clone https://github.com/ggerganov/llama.cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 32.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4503210 sha256=bf4deb0c027ce2f2ab22479467df25740f849c5536e57214cb8cc7117fcad59a
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 71266, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 71266 (delta 148), reused 86 (delta 86), pack-reused 71016 (from 4)
Receiving objects: 100% (71266/71266), 230.19 MiB | 18.07 MiB/s, done.
Resolving del

In [15]:
!python llama.cpp/convert_hf_to_gguf.py \
    /kaggle/working/chess-qwen-adapted \
    --outfile /kaggle/working/chess-fp16.gguf \
    --outtype f16

Writing: 100%|███████████████████████████| 1.19G/1.19G [00:03<00:00, 314Mbyte/s]


In [16]:
!cd llama.cpp && mkdir -p build && cd build && cmake .. && make -j

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [17]:
!llama.cpp/build/bin/llama-quantize \
    /kaggle/working/chess-fp16.gguf \
    /kaggle/working/chess-q4_K_M.gguf \
    q4_K_M

main: build = 7336 (6b82eb788)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/kaggle/working/chess-fp16.gguf' to '/kaggle/working/chess-q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 29 key-value pairs and 310 tensors from /kaggle/working/chess-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_k i32              = 20
llama_model_loader: - kv   3:                     general.sampling.top_p f32              = 0.950000
llama_model_loader: - kv   4:                      general.sampling.temp f32              = 0.600000
llama_model_loader: - kv   5:                               general.name str              = 